In [1]:
!pip install razdel
!pip install transformers evaluate accelerate # векторизатор

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00


In [26]:
!pip install datasets

In [9]:
from razdel import tokenize, sentenize
import copy

In [19]:
import pandas as pd
file = input("Введите путь к файлу с отзывами")
df =  pd.read_csv(file, sep = "\t", names=["id", "text"], index_col = False)
df

Введите путь к файлу с отзывами/content/train_reviews.txt


,id,text
0,3976,"День 8-го марта прошёл, можно и итоги подвести..."
1,30808,Отмечали в этом ресторане день рождение на пер...
2,14031,Хочу поделиться своим впечатлением от посещени...
3,2495,Добрый день! Были вчера с друзьями в этом кафе...
4,38835,Отметили с мужем годовщину свадьбы 6 ноября в ...
...,...,...
279,6962,Очаровательная Виктория просила об отзыве и я ...
280,9878,Пришли в данное заведение 4 июня 2014 года пок...
281,28258,Заехали с мужем поужинать в пятницу ( 17.01.14...
282,33043,Мне так там нравитсяяяя!!!!!!!!! Интерьер модн...


In [20]:
def clause_splitter(text):
    splitters = ':;,().…'

    conj = ['а', 'но', 'однако', 'чтобы', 'потому что', 'если',
        'несмотря на', 'хотя', 'так что', 'или', 'либо',]

    result = []
    for sent in list(sentenize(text)):
        s = sent.text.lower()
        parts = [s,]
        for splt in splitters:
            new_parts = []
            for part in parts:
                new_parts.extend(part.split(splt))
            parts = copy.deepcopy(new_parts)


        good_parts = []
        for part in parts:
            part = part.strip(' ')
            if len(part) > 5:
                if len(part.split(' ')) < 8:
                    good_parts.append(part)
                else:
                    splitted = False
                    for c in conj:
                        if f' {c} ' in part and not part.startswith(f'{c} '):
                            good_parts.extend(part.split(f' {c} '))
                            splitted = True
                            break
                    if not splitted:
                        good_parts.append(part)
        # наконец, место в тексте
        for gp in good_parts:
            if len(gp) > 5:
                result.append(gp.strip(' '))
    return result

In [21]:
df['text'] = df['text'].apply(clause_splitter)
df

,id,text
0,3976,"[день 8-го марта прошёл, можно и итоги подвест..."
1,30808,[отмечали в этом ресторане день рождение на пе...
2,14031,[хочу поделиться своим впечатлением от посещен...
3,2495,"[добрый день!, были вчера с друзьями в этом ка..."
4,38835,[отметили с мужем годовщину свадьбы 6 ноября в...
...,...,...
279,6962,[очаровательная виктория просила об отзыве и я...
280,9878,[пришли в данное заведение 4 июня 2014 года по...
281,28258,"[заехали с мужем поужинать в пятницу, первым н..."
282,33043,"[мне так там нравитсяяяя!!!!!!!!!, интерьер мо..."


In [22]:
# Применяем explode к столбцу "sentences"
df_exploded = df.explode('text')

In [25]:
df_exploded = df_exploded.drop("id", axis=1)
df_exploded

,text
0,день 8-го марта прошёл
0,можно и итоги подвести
0,решил написать отзыв о ресторане в котором отм...
0,прочитал отзывы edik077 и rules77777и понял что
0,мы были вразных ресторанах
...,...
283,могу сказать одно
283,я была приятно удивлена!
283,спасибо персоналу
283,ему удается создать ощущение


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=6, id2label=id2label, label2id=label2id
)
model.load_state_dict(torch.load('saved_weights_aspect.pt'))

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model=model)

classifier(df_exploded["text"])